<a href="https://colab.research.google.com/github/michelucci/zhaw-dlcourse-spring2019/blob/master/Additional%20Material%20-%20RNN/RNN%20Basic%20Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks and Deep Learning for Life Sciences and Health Applications - An introductory course about theoretical fundamentals, case studies and implementations in python and tensorflow

(C) Umberto Michelucci 2018 - umberto.michelucci@gmail.com 

github repository: https://github.com/michelucci/zhaw-dlcourse-spring2019

Spring Semester 2019

In [0]:
import numpy as np
import tensorflow as tf

In [0]:
from random import shuffle

# Explanation of data preparation

In [0]:
 ['{0:04b}'.format(i) for i in range(2**4)]

['0000',
 '0001',
 '0010',
 '0011',
 '0100',
 '0101',
 '0110',
 '0111',
 '1000',
 '1001',
 '1010',
 '1011',
 '1100',
 '1101',
 '1110',
 '1111']

# Data preparation

In [0]:
nn = 15
ll = 2**15

In [0]:
train_input = ['{0:015b}'.format(i) for i in range(ll)]
shuffle(train_input)
train_input = [map(int,i) for i in train_input]
ti  = []
for i in train_input:
    temp_list = []
    for j in i:
            temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti

In [0]:
np.asarray(train_input).shape

(32768, 15, 1)

In [0]:
np.asarray(train_input).reshape(ll,nn)

array([[0, 1, 1, ..., 0, 0, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 0, ..., 1, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 1, 1, 1]])

In [0]:
train_output = []
 
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*(nn+1))
    temp_list[count]=1
    train_output.append(temp_list)

In [0]:
np.asarray(train_output).shape

(32768, 16)

In [0]:
NUM_EXAMPLES = ll-2000
test_input = train_input[NUM_EXAMPLES:]
test_output = train_output[NUM_EXAMPLES:] #everything beyond 10,000
 
train_input = train_input[:NUM_EXAMPLES]
train_output = train_output[:NUM_EXAMPLES] #till 10,000

In [0]:
np.asarray(test_input).shape

(2000, 15, 1)

In [0]:
np.asarray(train_input).shape

(30768, 15, 1)

In [0]:
tf.reset_default_graph()

data = tf.placeholder(tf.float32, [None, nn,1])
target = tf.placeholder(tf.float32, [None, (nn+1)])

num_hidden = 24
cell = tf.nn.rnn_cell.LSTMCell(num_hidden,state_is_tuple=True)

val, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)
val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)

weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))

prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)

In [0]:
cross_entropy = -tf.reduce_sum(target * tf.log(tf.clip_by_value(prediction,1e-10,1.0)))

In [0]:
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

C:\Users\umber\Anaconda3\envs\tensorflow Keras\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

In [0]:
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

In [0]:
len(train_input)

30768

In [0]:
batch_size = 1000
no_of_batches = int(len(train_input)/batch_size)
epoch = 100
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        sess.run(minimize,{data: inp, target: out})
    if (i % 10 == 0):
        print ("Epoch - ",str(i))
        incorrect = sess.run(error,{data: test_input, target: test_output})
        print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))
incorrect = sess.run(error,{data: test_input, target: test_output})
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))
#sess.close()

Epoch -  0
Epoch  1 error 89.3%
Epoch -  10
Epoch 11 error 29.3%
Epoch -  20
Epoch 21 error 16.0%
Epoch -  30
Epoch 31 error 8.6%
Epoch -  40
Epoch 41 error 3.7%
Epoch -  50
Epoch 51 error 2.9%
Epoch -  60
Epoch 61 error 2.6%
Epoch -  70
Epoch 71 error 2.0%
Epoch -  80
Epoch 81 error 1.9%
Epoch -  90
Epoch 91 error 1.7%
Epoch 100 error 1.4%


In [0]:
print (sess.run(tf.argmax(prediction, 1),{data: [[[1],[0],[0],[1],[1],[0],[1],[1],[1],[0],[1],[0],[0],[1],[1]]]}))

[9]


# Normal network (that does not learn)

In [0]:
n_dim = 15
tf.reset_default_graph()

# Number of neurons in the layers
n1 = 15 # Number of neurons in layer 1
n2 = 16 # Number of neurons in output layer 

cost_history = np.empty(shape=[1], dtype = float)
learning_rate = tf.placeholder(tf.float32, shape=())

X = tf.placeholder(tf.float32, [15, None])
Y = tf.placeholder(tf.float32, [16, None])
W1 = tf.Variable(tf.truncated_normal([n1, n_dim], stddev=.1)) 
b1 = tf.Variable(tf.constant(0.1, shape = [n1,1]) )
W2 = tf.Variable(tf.truncated_normal([n2, n1], stddev=.1)) 
b2 = tf.Variable(tf.constant(0.1, shape = [n2,1])) 
                 
# Let's build our network...
Z1 = tf.nn.relu(tf.matmul(W1, X) + b1) # n1 x n_dim * n_dim x n_obs = n1 x n_obs
Z2 = tf.matmul(W2, Z1) + b2 # n2 x n1 * n1 * n_obs = n2 x n_obs
y_ = tf.nn.softmax(Z2,0) # n2 x n_obs (10 x None)


prediction = tf.nn.softmax(y_)

cost = - tf.reduce_mean(Y * tf.log(y_)+(1-Y) * tf.log(1-y_))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

mistakes = tf.not_equal(tf.argmax(Y, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

init = tf.global_variables_initializer()

In [0]:
train_size = 30768
test_size = 2000

sess = tf.Session()
sess.run(tf.global_variables_initializer())

training_epochs = 20
    
cost_history = []
for epoch in range(training_epochs+1):

    sess.run(optimizer, feed_dict = {X: np.asarray(train_input).reshape(15,30768), 
                                     Y: np.asarray(train_output).reshape(16,30768), learning_rate: .1})
    cost_ = sess.run(cost, feed_dict={ X:np.asarray(train_input).reshape(15,30768), 
                                      Y: np.asarray(train_output).reshape(16,30768), learning_rate: .1})
    cost_history = np.append(cost_history, cost_)
    
    if (epoch % 10 == 0):
        print("Reached epoch",epoch,"cost J =", cost_)
        incorrect = sess.run(error,{X: np.asarray(test_input).reshape(15,test_size), 
                                    Y: np.asarray(test_output).reshape(16,test_size)})
        print(100 * incorrect)

Reached epoch 0 cost J = 0.23398738
100.0
Reached epoch 10 cost J = 0.23398362
100.0
Reached epoch 20 cost J = 0.23398054
100.0


In [0]:
np.asarray(train_input).reshape(15,30768)

array([[0, 1, 1, ..., 0, 1, 1],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 1, 1, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 1],
       [0, 1, 0, ..., 1, 0, 1],
       [0, 1, 1, ..., 0, 1, 0]])